##### Libraries

In [9]:
def loadLibraries():
    import os
    import json
    import datetime as dt
    import pytz
    from google.oauth2.credentials import Credentials
    from google_auth_oauthlib.flow import InstalledAppFlow
    from google.auth.transport.requests import Request
    from googleapiclient.discovery import build
    from googleapiclient.errors import HttpError
    SCOPES = [
    'https://www.googleapis.com/auth/calendar',
    'https://www.googleapis.com/auth/tasks'
    ]

In [10]:
import os
import json
import datetime as dt
import pytz
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
SCOPES = [
    'https://www.googleapis.com/auth/calendar',
    'https://www.googleapis.com/auth/tasks'
]

#### Fetching Cradentials

In [ ]:
def get_credentials():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    return creds

#### Adding Events to calendar

In [5]:
def add_event(event_details):
    creds = get_credentials()
    try:
        service = build('calendar', 'v3', credentials=creds)
        
        end_time_str = event_details.get('Deadline')
        end_time = dt.datetime.fromisoformat(end_time_str)
        
        tz = pytz.timezone("Asia/Kolkata")
        start_time = dt.datetime.now(tz)
        
        if end_time.tzinfo is None:
            end_time = tz.localize(end_time)
        else:
            end_time = end_time.astimezone(tz)
        if end_time <= start_time:
            print(f"Skipping event '{event_details.get('Task')}' because the deadline is not in the future.")
            return
        
        event = {
            'summary': event_details.get('Task'),
            'description': f"Responsible Person: {event_details.get('Responsible Person')}",
            'start': {
                'dateTime': start_time.isoformat(),
                'timeZone': 'Asia/Kolkata',
            },
            'end': {
                'dateTime': end_time.isoformat(),
                'timeZone': 'Asia/Kolkata',
            }
        }
        
        created_event = service.events().insert(calendarId='primary', body=event).execute()
        print('Event created: %s' % (created_event.get('htmlLink')))
    except HttpError as e:
        print(e)

#### Adding Tasks to Google Task

In [11]:
def add_task(task_details):
    creds = get_credentials()
    try:
        service = build('tasks', 'v1', credentials=creds)
        
        # Construct a task. If there's a deadline, include it; otherwise, skip it.
        task_body = {
            'title': task_details.get('Task'),
            'notes': f"Responsible Person: {task_details.get('Responsible Person')}"
        }
        # Optional: include due date if deadline exists and is in the future.
        deadline = task_details.get('Deadline')
        if deadline:
            due_time = dt.datetime.fromisoformat(deadline)
            tz = pytz.timezone("Asia/Kolkata")
            if due_time.tzinfo is None:
                due_time = tz.localize(due_time)
            else:
                due_time = due_time.astimezone(tz)
            now = dt.datetime.now(tz)
            if due_time > now:
                # Google Tasks API expects the due date in RFC3339 format (only date part is required for tasks)
                task_body['due'] = due_time.isoformat()
        
        result = service.tasks().insert(tasklist='@default', body=task_body).execute()
        print(f"Task added: {result.get('title')}")
    except HttpError as e:
        print(e)

In [12]:
def addTasks(data):
    loadLibraries()

    for event_item in data.get("Action Items & Tasks", []):
        if event_item.get('Deadline') is not None:
            add_event(event_item)

    for task_item in data.get("Action Items & Tasks", []):
        add_task(task_item)